In [2]:
import pandas as pd
# Đường dẫn đến tệp Excel
file_path = 'D:\Báo cáo tốt nghiệp\Check.xlsx'

# Đọc dữ liệu từ tệp Excel
df = pd.read_excel(file_path)
print(df)

<>:3: SyntaxWarning: invalid escape sequence '\B'
<>:3: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Admin\AppData\Local\Temp\ipykernel_25868\1294996154.py:3: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'D:\Báo cáo tốt nghiệp\Check.xlsx'


           Lớp    MSV Mã học phần                        Tên học phần  \
0      46K23.1      1     HOS3010  An ninh và an toàn trong khách sạn   
1      46K23.1      2     HOS3010  An ninh và an toàn trong khách sạn   
2      46K23.1      3     HOS3010  An ninh và an toàn trong khách sạn   
3      46K23.1      4     HOS3010  An ninh và an toàn trong khách sạn   
4      46K23.1      5     HOS3010  An ninh và an toàn trong khách sạn   
...        ...    ...         ...                                 ...   
47999  47K13.2  12609     LAW3040          Xây dựng văn bản pháp luật   
48000  47K13.2  12610     LAW3040          Xây dựng văn bản pháp luật   
48001  47K13.2  12611     LAW3040          Xây dựng văn bản pháp luật   
48002  47K13.2   2274     LAW3040          Xây dựng văn bản pháp luật   
48003  47K13.2  12432     LAW3040          Xây dựng văn bản pháp luật   

      Đề thi TA  Số tín chỉ     Lớp học phần Loại đăng ký  HP miễn ngoại ngữ  \
0           NaN           2  HOS3010_46K23.

In [3]:
edges = set()
grouped=df.groupby('MSV')
for _, group in grouped:
    subject=group['Mã học phần'].tolist()
    edges.update([(subject[i], subject[j]) for i in range(len(subject)) for j in range(i+1, len(subject))])

    

In [29]:
import numpy
import pandas as pd
import math

def assign_exam_schedule(df, max_shifts, max_rooms, students_per_room):
    course_students = df.groupby('Mã học phần')['MSV'].count()
    course_rooms = dict([[kcs, math.ceil(course_students[kcs]/students_per_room)] for kcs in course_students.keys()])

    students_per_shift = students_per_room*max_rooms
    shifts = {}
    df['Ca thi'] = ""
    # Sắp xếp các mã học phần theo slsv đăng ký
    sorted_nodes = course_students.sort_values(ascending=False)['Mã học phần'].tolist()     
    #Tạo các cạnh nối giữa các mã học phần
    edges = set()

    # Xem sinh viên đó học bao nhiêu học phần 
    for _, group in df.groupby('MSV'): 
        #       
        subjects = group['Mã học phần'].tolist()
        
        #Tạo ra các cặp cạnh i,j trong danh sách subject. Các cặp này sẽ tạo thành các cạnh biễu diễn bằng giá trị i,j
        edges.update([(subjects[i], subjects[j]) for i in range(len(subjects)) for j in range(i+1, len(subjects))])

    i = 1    
    while sorted_nodes <=0:
        # Chọn đỉnh đầu tiên trong danh sách V
        start_node = sorted_nodes[0]
        
        # Tô màu i cho đỉnh đang xét
        #Số lượng sinh viên của mã học phần đầu tiên
        no_students = len(df.loc[df['Mã học phần'] == start_node])
 
        if no_students > students_per_shift:
            if i not in shifts: 
                no_required_shifts = math.ceil(no_students/students_per_shift)
                no_node_students_per_shift = math.ceil(no_students/no_required_shifts)
                j = 0
                for k,r in df.loc[df['Mã học phần'] == start_node].iterrows():
                    df.loc[k,'Ca thi'] = i+int(j/no_node_students_per_shift)
                    j += 1
                
                for j in range(no_required_shifts):
                    shifts[i+j] += math.ceil(no_node_students_per_shift/students_per_room)
                    
                sorted_nodes = sorted_nodes[1:]
                
        elif no_students < students_per_shift:
            #Lọc ra các học phần còn lại
            unassigned_course = df[df['Ca thi'].isnull()]['Mã học phần'].unique()
            new_edges = set()   
        
            for _, group in df[df['Mã học phần'].isin(unassigned_course)].groupby('MSV'):
            # Tạo danh sách chứa các mã học phần trong nhóm hiện tại
                subjects = group['Mã học phần'].tolist()
            # Tạo các cặp cạnh giữa các mã học phần trong nhóm
                new_edges.update([(subjects[i], subjects[j]) for i in range(len(subjects)) for j in range(i + 1, len(subjects))])
            # Tạo từ điển để lưu trữ số cạnh của mỗi node
            node_edge_counts = {}
            #if 'ENG3008' in node_edge_counts:
            #print(node_edge_counts['ENG3008'])
            for edge in new_edges:
                for node in edge:
                    # Tăng số lượng cạnh cho node lên 1
                    if node in node_edge_counts:
                        node_edge_counts[node] += 1
                    else:       
                        node_edge_counts[node] = 1
            # Sắp xếp các cạnh của mỗi node theo thứ tự giảm dần
            sorted_nodes_by_edge_count = sorted(node_edge_counts.keys(), key=lambda x: node_edge_counts[x], reverse=True)
            if len(sorted_nodes_by_edge_count)>0:   
                first_node=sorted_nodes_by_edge_count[0]
               
                df.loc[df['Mã học phần'] == first_node, 'Ca thi'] = i
                
                sorted_nodes_by_edge_count = sorted_nodes_by_edge_count[1:]

                # sorted_nodes = sorted_nodes[1:]
             
        jn = 0
        while jn < len(sorted_nodes):
            node =  sorted_nodes[jn]
            #Danh sách các đỉnh đã được tô màu i
            colored_nodes = set(df[df['Ca thi'] == i]['Mã học phần'].tolist())
            if all((node, colored_node) not in edges and (colored_node, node) not in edges for colored_node in colored_nodes):
                # Nếu không kề với các học phần đã tô màu i, tô màu i cho đỉnh đó
                remaining_room=max_rooms-shifts[i]
                if course_rooms[node] <= remaining_room:                
                    df.loc[df['Mã học phần'] == node, 'Ca thi'] = i
                    shifts[i] += course_rooms[node]
                    #uncolored_nodes.remove(node)
                    sorted_nodes = sorted_nodes[:jn] + sorted_nodes[jn+1:]
                    #colored_nodes += [node]
                    jn -= 1
            else:
                print(node)
            jn += 1
     
        # Tăng giá trị màu i lên 1 đơn vị
        i += 1
# Gọi hàm để gán ca thi
MAX_SHIFTS = 60
MAX_ROOMS = 35
STUDENTS_PER_ROOM = 40
assign_exam_schedule(df, MAX_SHIFTS, MAX_ROOMS, STUDENTS_PER_ROOM)

TOU1001
ACC1002
SMT1007
MIS2002
ENG2015
SMT1004
SMT1008
FIN2001
ENG3001
SMT1006
HRM2001
IBS2001
MGT2002
FIN3004
ACC3004
LAW2001
MGT3004
FIN3002
HOS3001
ENGELE2
ACC3001
MGT3002
FIN3001
MIS2001
IBS3013
IBS3007
AUD3004
HRM3005
IBS3001
MGT3007
IBS3002
LAW3001
ENG3005
BAN3014
STA3001
ACC3008
ENG2016
BAN2001
ENG2017
ACC3002
IBS3004
FIN3003
ECO3001
MGT1002
ECO2001
MIS3006
MGT3003
SMT3010
LAW2007
TOU3002
ECO2004
BAN3002
SMT1005
IBS3010
BAN3003
LAW3042
MGT3009
BAN3015
ELC3008
BAN3005
AUD2001
IBS2003
HRM3010
MIS3024
MKT3008
IBS3003
IBS3014
ECO3002
MKT3004
LAW3059
MGT3020
BAN3006
HOS3019
ECO3024
HOS3021
TOU3056
MKT3007
BAN3007
ACC3015
LAW3006
TOU3048
HRM3004
ECO3015
HRM3009
ECO3007
IBS3008
ACC2003
TOU3051
ACC3007
SMT3019
ECO1001
SMT3033
SMT3027
TOU3060
SMT3002
ECO2002
ECO3041
IBS3011
FIN3005
SMT2001
ECO3036
ECO2003
SMT3020
SMT3018
SMT3025
SMT3003
ECO3022
MIS1002
IBS3005
MAT1001
SMT3001
SMT3028
LAW1001
VNA1002
VNA2004
MKT2001
FIN5009U
TOU1001
ACC1002
SMT1007
MIS2002
ENG2015
SMT1004
SMT1008
FIN2001

In [30]:


# # Loại bỏ các dòng trùng lặp khỏi DataFrame gốc
# df = df.drop_duplicates(subset=['MSV mở rộng', 'Mã học phần mở rộng'], keep='first')




In [31]:
# Nhóm dữ liệu theo cột 'MSV' và 'Ca thi', sau đó đếm số lượng dòng trong mỗi nhóm
duplicate_schedule = df.groupby(['MSV mở rộng', 'Mã học phần mở rộng']).size().reset_index(name='Count')

# Lọc ra các nhóm có số lượng dòng lớn hơn 1, tức là các MSV bị trùng lịch thi
duplicated_students = duplicate_schedule[duplicate_schedule['Count'] > 1]   

# print(df_cleaned)
if not duplicated_students.empty:
    print("Có MSV bị trùng lịch thi:")
    print(duplicated_students)
else:
    print("Không có MSV nào bị trùng lịch thi.")

Không có MSV nào bị trùng lịch thi.


In [32]:
df.to_excel('D:\Báo cáo tốt nghiệp\Kiem tra duplicate.xlsx', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Admin\AppData\Local\Temp\ipykernel_15216\3326509927.py:1: SyntaxWarning: invalid escape sequence '\B'
  df.to_excel('D:\Báo cáo tốt nghiệp\Kiem tra duplicate.xlsx', index=False)
